In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Federated Project/FL-project

/content/drive/MyDrive/Federated Project/FL-project


In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from main import *
import argparse
from centralized import Centralized
from client import *
import wandb

In [5]:
class Args():
  def __init__(self, dataset, model, bs, hnm, num_epochs):
    self.dataset = dataset
    self.model = model 
    self.bs = bs
    self.hnm = hnm
    self.num_epochs = num_epochs

args = Args('iddaCB', model = 'deeplabv3_mobilenetv2', bs = 4, hnm = False, num_epochs = 5)

In [6]:
trainAloneDataset, testAloneDatasets = get_datasets(args)
ds = trainAloneDataset[0]

In [7]:
#istanziamo il modello 
model = model_init(args)

In [9]:
centralized = Centralized(args,ds,model)
state_dict = torch.load('modelliSalvati/checkpoint.pth')
model.classifier.load_state_dict(state_dict)
centralized.train2()

params freezed
epoch 0
epoch 1 / 5, step 1 / 150, loss = 0.633
epoch 1 / 5, step 2 / 150, loss = 0.660
epoch 1 / 5, step 3 / 150, loss = 0.604
epoch 1 / 5, step 4 / 150, loss = 0.713
epoch 1 / 5, step 5 / 150, loss = 0.495
epoch 1 / 5, step 6 / 150, loss = 0.709
epoch 1 / 5, step 7 / 150, loss = 0.761
epoch 1 / 5, step 8 / 150, loss = 0.655
epoch 1 / 5, step 9 / 150, loss = 0.607
epoch 1 / 5, step 10 / 150, loss = 0.549
epoch 1 / 5, step 11 / 150, loss = 0.659
epoch 1 / 5, step 12 / 150, loss = 0.618
epoch 1 / 5, step 13 / 150, loss = 0.657
epoch 1 / 5, step 14 / 150, loss = 0.788
epoch 1 / 5, step 15 / 150, loss = 0.592
epoch 1 / 5, step 16 / 150, loss = 0.634
epoch 1 / 5, step 17 / 150, loss = 0.715
epoch 1 / 5, step 18 / 150, loss = 0.650
epoch 1 / 5, step 19 / 150, loss = 0.662
epoch 1 / 5, step 20 / 150, loss = 0.837
epoch 1 / 5, step 21 / 150, loss = 0.580
epoch 1 / 5, step 22 / 150, loss = 0.557
epoch 1 / 5, step 23 / 150, loss = 0.689
epoch 1 / 5, step 24 / 150, loss = 0.742
ep

KeyboardInterrupt: ignored

In [ ]:
from utils.stream_metrics import StreamSegMetrics
metric = StreamSegMetrics(16, 'centralizzato')
centralized.test(metric)

In [ ]:
metric.get_results()['Mean IoU']

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(ds, batch_size = 2, shuffle = False, drop_last = True)

In [ ]:
centralized.model.eval()
with torch.no_grad():
  for i, (images, labels) in enumerate(train_loader):
    images = images.cuda() 
    labels = labels.cuda()
    outputs = centralized.model(images)['out']
    break

In [ ]:
outputs = centralized.model(images[0].view(1, 3, 512, 928))['out']

In [ ]:
import matplotlib.pyplot as plt
def unNormalize(tensorImage, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        for t, m, s in zip(tensorImage, mean, std):
            t.mul_(s).add_(m)
        return tensorImage
plt.imshow(unNormalize(images[0].cpu()).permute(1,2,0))

In [ ]:
singoloOutputLogits = outputs[0]
prediction = singoloOutputLogits.argmax(0)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(unNormalize(images[0].cpu()).permute(1,2,0))
plt.imshow(prediction.cpu().numpy(), alpha = 0.4)

In [ ]:
import matplotlib as mpl

plt.imshow(labels[0].cpu().numpy(), cmap = mpl.colormaps['inferno'], vmin = 0, vmax = 16)